In [2]:
!pip3 install -q -U 'sagemaker>=2.126.0'

In [3]:
!pip3 install autogluon --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [4]:
# Required imports
import sagemaker
import pandas as pd
import os
import boto3
from sagemaker import utils
from sagemaker.serializers import CSVSerializer

In [5]:
# Helper wrappers
from ag_model import (
    AutoGluonSagemakerEstimator,
    AutoGluonNonRepackInferenceModel,
    AutoGluonSagemakerInferenceModel,
    AutoGluonRealtimePredictor,
)

# Sagemaker variables 
role = sagemaker.get_execution_role()
session = sagemaker.Session()
region = session._region_name

bucket = "mle-capstone"
s3_prefix = f"{utils.sagemaker_timestamp()}"
s3_data_prefix = "final-data/Final-Capstone-Data.zip"
output_path = f"s3://{bucket}/{s3_prefix}/output/"

In [38]:
# Command to download data from S3 (This command needs to be executed once)
session.download_data(path="data", bucket=bucket, key_prefix=s3_data_prefix)

In [39]:
# Command to unzip the data (This command needs to be executed once)
!unzip -o data/Final-Capstone-Data.zip

Archive:  data/Final-Capstone-Data.zip
   creating: Final-Capstone-Data/
  inflating: Final-Capstone-Data/valid.csv  
  inflating: Final-Capstone-Data/test.csv  
  inflating: Final-Capstone-Data/train.csv  


In [6]:
# Training job Estimator
ag = AutoGluonSagemakerEstimator(
    role=role,
    entry_point="scripts/training_script.py",
    region=region,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    framework_version="0.7",
    py_version="py39",
    base_job_name="autogluon-train",
    disable_profiler=True,
    debugger_hook_config=False,
)

In [47]:
train_input = ag.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "train.csv"), bucket=bucket, key_prefix=s3_prefix
)
valid_input = ag.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "valid.csv"), bucket=bucket, key_prefix=s3_prefix
)
eval_input = ag.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "test.csv"), bucket=bucket, key_prefix=s3_prefix
)
config_input = ag.sagemaker_session.upload_data(
    path=os.path.join("config", "config-med.yaml"), bucket=bucket, key_prefix=s3_prefix
)
inference_script = ag.sagemaker_session.upload_data(
    path=os.path.join("scripts", "inference_script.py"), bucket=bucket, key_prefix=s3_prefix
)

job_name = utils.unique_name_from_base("test-autogluon-image")
ag.fit(
    {
        "config": config_input,
        "train": train_input,
        "valid": valid_input,
        "test": eval_input,
        "serving": inference_script
    },
    job_name=job_name,
)

INFO:sagemaker:Creating training-job with name: test-autogluon-image-1681329605-8167


2023-04-12 20:00:09 Starting - Starting the training job...
2023-04-12 20:00:24 Starting - Preparing the instances for training...
2023-04-12 20:01:14 Downloading - Downloading input data...
2023-04-12 20:01:34 Training - Downloading the training image.........
2023-04-12 20:03:05 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-04-12 20:03:33,954 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-04-12 20:03:33,956 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-12 20:03:33,958 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-12 20:03:33,968 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-04-12 20:03:33,970 sagemaker_pytorch_container.training INFO     Invoking user trai

# Deployment

In [50]:
# Download the trained model saved in S3 (To be executed once)
!aws s3 cp {ag.model_data} .

download: s3://sagemaker-us-east-1-004538843871/test-autogluon-image-1681329605-8167/output/model.tar.gz to ./model.tar.gz


In [51]:
!ls -alF model.tar.gz

-rw-rw-r-- 1 ec2-user ec2-user 406665479 Apr 12 20:16 model.tar.gz


In [52]:
# Upload the latest trained model to a desired path in S3
endpoint_name = sagemaker.utils.unique_name_from_base("autogluon-serving-trained-model")

model_data = session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

In [22]:
# Activate to attach previously uploaded model data and don't execute previous cells under deployment
#model_data = 's3://sagemaker-us-east-1-004538843871/autogluon-serving-trained-model-1681331142-fa4d/models/model.tar.gz'

In [23]:
# Configure for deployment and inferencing
instance_type = "ml.m5.2xlarge"

model = AutoGluonNonRepackInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    framework_version="0.7",
    py_version="py39",
    instance_type=instance_type,
    source_dir="scripts",
    entry_point="inference_script.py"
)

In [24]:
# Deploy the model
model.deploy(initial_instance_count=1, serializer=CSVSerializer(), instance_type=instance_type)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating model with name: autogluon-inference-2023-04-15-19-02-32-311
INFO:sagemaker:Creating endpoint-config with name autogluon-inference-2023-04-15-19-02-32-955
INFO:sagemaker:Creating endpoint with name autogluon-inference-2023-04-15-19-02-32-955


------!

In [25]:
# Print deployed latest endpoint details
model.endpoint_name

'autogluon-inference-2023-04-15-19-02-32-955'

In [26]:
# Create a predictor for testing without Lambda
predictor = AutoGluonRealtimePredictor(model.endpoint_name)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [27]:
# Load Test data
df_test = pd.read_csv("Final-Capstone-Data/test.csv")

In [28]:
# Predict on test data by removing columns not required
preds = predictor.predict(df_test.drop(columns=["customer_ID", "S_2", "target"], axis=1))

In [29]:
# Combine predicted and actual values in same Dataframe
p = preds[["pred"]]
p = p.join(df_test["target"]).rename(columns={"target": "actual"})
p.head()

,pred,actual
0,1,1
1,1,1
2,0,0
3,1,1
4,1,1


In [30]:
# Print the accurate prediction count
print(f"{(p.pred==p.actual).astype(int).sum()}/{len(p)} are correct")

4982/6000 are correct


In [31]:
print(f"{p[(p['pred']==1) & (p['actual']==1)].shape[0]}/1500 defaulters are correct")

1355/1500 defaulters are correct


In [32]:
print(f"{p[(p['pred']==0) & (p['actual']==0)].shape[0]}/4500 non-defaulters are correct")

3627/4500 non-defaulters are correct


# Hyperparameter Optimization

In [9]:
train_input = ag.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "train.csv"), bucket=bucket, key_prefix=s3_prefix
)
valid_input = ag.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "valid.csv"), bucket=bucket, key_prefix=s3_prefix
)
eval_input = ag.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "test.csv"), bucket=bucket, key_prefix=s3_prefix
)
config_input = ag.sagemaker_session.upload_data(
    path=os.path.join("config", "config-hpo.yaml"), bucket=bucket, key_prefix=s3_prefix
)
inference_script = ag.sagemaker_session.upload_data(
    path=os.path.join("scripts", "inference_script.py"), bucket=bucket, key_prefix=s3_prefix
)

job_name = utils.unique_name_from_base("test-autogluon-image")
hpo_predictor = ag.fit(
    {
        "config": config_input,
        "train": train_input,
        "valid": valid_input,
        "test": eval_input,
        "serving": inference_script
    },
    job_name=job_name,
)

INFO:sagemaker:Creating training-job with name: test-autogluon-image-1681570642-16bb


2023-04-15 14:57:23 Starting - Starting the training job...
2023-04-15 14:57:38 Starting - Preparing the instances for training...
2023-04-15 14:58:18 Downloading - Downloading input data...
2023-04-15 14:58:38 Training - Downloading the training image......
2023-04-15 14:59:49 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-04-15 15:00:11,563 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-04-15 15:00:11,565 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-15 15:00:11,567 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-15 15:00:11,577 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-04-15 15:00:11,579 sagemaker_pytorch_container.training INFO     Invoking user trainin

In [13]:
type(hpo_predictor)

NoneType

In [14]:
# Download the trained model saved in S3 (To be executed once)
!aws s3 cp {ag.model_data} .

download: s3://sagemaker-us-east-1-004538843871/test-autogluon-image-1681570642-16bb/output/model.tar.gz to ./model.tar.gz


In [15]:
!ls -alF model.tar.gz

-rw-rw-r-- 1 ec2-user ec2-user 555762994 Apr 15 15:10 model.tar.gz


In [16]:
# Upload the latest trained model to a desired path in S3
endpoint_name = sagemaker.utils.unique_name_from_base("autogluon-serving-trained-model")
print(endpoint_name)

autogluon-serving-trained-model-1681572062-df69


In [17]:
model_data = session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

In [18]:
# Configure for deployment and inferencing
instance_type = "ml.m5.2xlarge"

model = AutoGluonNonRepackInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    framework_version="0.7",
    py_version="py39",
    instance_type=instance_type,
    source_dir="scripts",
    entry_point="inference_script.py"
)

In [19]:
# Deploy the model
model.deploy(initial_instance_count=1, serializer=CSVSerializer(), instance_type=instance_type)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating model with name: autogluon-inference-2023-04-15-15-22-54-164
INFO:sagemaker:Creating endpoint-config with name autogluon-inference-2023-04-15-15-22-54-985
INFO:sagemaker:Creating endpoint with name autogluon-inference-2023-04-15-15-22-54-985


-----!

In [20]:
# Print deployed latest endpoint details
model.endpoint_name

'autogluon-inference-2023-04-15-15-22-54-985'

In [21]:
# Create a predictor for testing without Lambda
predictor = AutoGluonRealtimePredictor(model.endpoint_name)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [22]:
# Load Test data
df_test = pd.read_csv("Final-Capstone-Data/test.csv")

In [23]:
# Predict on test data by removing columns not required
preds = predictor.predict(df_test.drop(columns=["customer_ID", "S_2", "target"], axis=1))

In [24]:
# Combine predicted and actual values in same Dataframe
p = preds[["pred"]]
p = p.join(df_test["target"]).rename(columns={"target": "actual"})
p.head()

,pred,actual
0,1,1
1,1,1
2,0,0
3,1,1
4,1,1


In [25]:
# Print the accurate prediction count
print(f"{(p.pred==p.actual).astype(int).sum()}/{len(p)} are correct")

4994/6000 are correct


In [41]:
print(f"{p[(p['pred']==1) & (p['actual']==1)].shape[0]}/1500 defaulters are correct")

1354/1500 defaulters are correct


In [44]:
print(f"{p[(p['pred']==0) & (p['actual']==0)].shape[0]}/4500 non-defaulters are correct")

3640/4500 non-defaulters are correct


# Hyperparameter Optimization 2

In [12]:
# Training job Estimator
ag = AutoGluonSagemakerEstimator(
    role=role,
    entry_point="scripts/training_script_2.py",
    region=region,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    framework_version="0.7",
    py_version="py39",
    base_job_name="autogluon-train",
    disable_profiler=True,
    debugger_hook_config=False,
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [13]:
train_input = ag.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "train.csv"), bucket=bucket, key_prefix=s3_prefix
)
valid_input = ag.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "valid.csv"), bucket=bucket, key_prefix=s3_prefix
)
eval_input = ag.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "test.csv"), bucket=bucket, key_prefix=s3_prefix
)
config_input = ag.sagemaker_session.upload_data(
    path=os.path.join("config", "config-hpo-2.yaml"), bucket=bucket, key_prefix=s3_prefix
)
inference_script = ag.sagemaker_session.upload_data(
    path=os.path.join("scripts", "inference_script.py"), bucket=bucket, key_prefix=s3_prefix
)

job_name = utils.unique_name_from_base("test-autogluon-image")
hpo_predictor = ag.fit(
    {
        "config": config_input,
        "train": train_input,
        "valid": valid_input,
        "test": eval_input,
        "serving": inference_script
    },
    job_name=job_name,
)

INFO:sagemaker:Creating training-job with name: test-autogluon-image-1681596542-61c6


2023-04-15 22:09:03 Starting - Starting the training job...
2023-04-15 22:09:20 Starting - Preparing the instances for training...
2023-04-15 22:10:02 Downloading - Downloading input data...
2023-04-15 22:10:22 Training - Downloading the training image......
2023-04-15 22:11:38 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-04-15 22:12:01,235 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-04-15 22:12:01,237 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-15 22:12:01,239 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-15 22:12:01,250 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-04-15 22:12:01,252 sagemaker_pytorch_container.training INFO     Invoking user trainin

In [40]:
# Download the trained model saved in S3 (To be executed once)
!aws s3 cp {ag.model_data} .

download: s3://sagemaker-us-east-1-004538843871/test-autogluon-image-1681596542-61c6/output/model.tar.gz to ./model.tar.gz


In [42]:
# Upload the latest trained model to a desired path in S3
endpoint_name = sagemaker.utils.unique_name_from_base("autogluon-serving-trained-model")
print(endpoint_name)

autogluon-serving-trained-model-1681601803-4f5d


In [43]:
model_data = session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

In [44]:
# Configure for deployment and inferencing
instance_type = "ml.m5.2xlarge"

model = AutoGluonNonRepackInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    framework_version="0.7",
    py_version="py39",
    instance_type=instance_type,
    source_dir="scripts",
    entry_point="inference_script.py"
)

In [45]:
# Deploy the model
model.deploy(initial_instance_count=1, serializer=CSVSerializer(), instance_type=instance_type)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating model with name: autogluon-inference-2023-04-15-23-36-50-675
INFO:sagemaker:Creating endpoint-config with name autogluon-inference-2023-04-15-23-36-51-327
INFO:sagemaker:Creating endpoint with name autogluon-inference-2023-04-15-23-36-51-327


----!

In [46]:
# Print deployed latest endpoint details
model.endpoint_name

'autogluon-inference-2023-04-15-23-36-51-327'

In [47]:
# Create a predictor for testing without Lambda
predictor = AutoGluonRealtimePredictor(model.endpoint_name)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [48]:
# Load Test data
df_test = pd.read_csv("Final-Capstone-Data/test.csv")

In [49]:
# Predict on test data by removing columns not required
preds = predictor.predict(df_test.drop(columns=["customer_ID", "S_2", "target"], axis=1))

In [50]:
# Combine predicted and actual values in same Dataframe
p = preds[["pred"]]
p = p.join(df_test["target"]).rename(columns={"target": "actual"})
p.head()

,pred,actual
0,1,1
1,1,1
2,0,0
3,1,1
4,1,1


In [51]:
# Print the accurate prediction count
print(f"{(p.pred==p.actual).astype(int).sum()}/{len(p)} are correct")

5010/6000 are correct


In [52]:
print(f"{p[(p['pred']==1) & (p['actual']==1)].shape[0]}/1500 defaulters are correct")

1345/1500 defaulters are correct


In [53]:
print(f"{p[(p['pred']==0) & (p['actual']==0)].shape[0]}/4500 non-defaulters are correct")

3665/4500 non-defaulters are correct


# HPO 3

In [24]:
# Training job Estimator
ag_3 = AutoGluonSagemakerEstimator(
    role=role,
    entry_point="scripts/training_script.py",
    region=region,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    framework_version="0.7",
    py_version="py39",
    base_job_name="autogluon-train",
    disable_profiler=True,
    debugger_hook_config=False,
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [25]:
train_input = ag_3.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "train.csv"), bucket=bucket, key_prefix=s3_prefix
)
valid_input = ag_3.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "valid.csv"), bucket=bucket, key_prefix=s3_prefix
)
eval_input = ag_3.sagemaker_session.upload_data(
    path=os.path.join("Final-Capstone-Data", "test.csv"), bucket=bucket, key_prefix=s3_prefix
)
config_input = ag_3.sagemaker_session.upload_data(
    path=os.path.join("config", "config-hpo-3.yaml"), bucket=bucket, key_prefix=s3_prefix
)
inference_script = ag_3.sagemaker_session.upload_data(
    path=os.path.join("scripts", "inference_script.py"), bucket=bucket, key_prefix=s3_prefix
)

job_name = utils.unique_name_from_base("test-autogluon-image")
hpo_predictor = ag_3.fit(
    {
        "config": config_input,
        "train": train_input,
        "valid": valid_input,
        "test": eval_input,
        "serving": inference_script
    },
    job_name=job_name,
)

INFO:sagemaker:Creating training-job with name: test-autogluon-image-1681600385-d696


2023-04-15 23:13:06 Starting - Starting the training job...
2023-04-15 23:13:20 Starting - Preparing the instances for training...
2023-04-15 23:14:05 Downloading - Downloading input data...
2023-04-15 23:14:25 Training - Downloading the training image......
2023-04-15 23:15:41 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-04-15 23:16:04,198 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-04-15 23:16:04,200 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-15 23:16:04,202 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-15 23:16:04,212 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-04-15 23:16:04,214 sagemaker_pytorch_container.training INFO     Invoking user trainin

In [26]:
# Download the trained model saved in S3 (To be executed once)
!aws s3 cp {ag_3.model_data} .

download: s3://sagemaker-us-east-1-004538843871/test-autogluon-image-1681600385-d696/output/model.tar.gz to ./model.tar.gz


In [28]:
# Upload the latest trained model to a desired path in S3
endpoint_name = sagemaker.utils.unique_name_from_base("autogluon-serving-trained-model")
print(endpoint_name)

autogluon-serving-trained-model-1681601198-dcae


In [29]:
model_data = session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

In [30]:
# Configure for deployment and inferencing
instance_type = "ml.m5.2xlarge"

model = AutoGluonNonRepackInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    framework_version="0.7",
    py_version="py39",
    instance_type=instance_type,
    source_dir="scripts",
    entry_point="inference_script.py"
)

In [31]:
# Deploy the model
model.deploy(initial_instance_count=1, serializer=CSVSerializer(), instance_type=instance_type)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating model with name: autogluon-inference-2023-04-15-23-26-47-133
INFO:sagemaker:Creating endpoint-config with name autogluon-inference-2023-04-15-23-26-47-758
INFO:sagemaker:Creating endpoint with name autogluon-inference-2023-04-15-23-26-47-758


-----!

In [32]:
# Print deployed latest endpoint details
model.endpoint_name

'autogluon-inference-2023-04-15-23-26-47-758'

In [33]:
# Create a predictor for testing without Lambda
predictor = AutoGluonRealtimePredictor(model.endpoint_name)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [34]:
# Load Test data
df_test = pd.read_csv("Final-Capstone-Data/test.csv")

In [35]:
# Predict on test data by removing columns not required
preds = predictor.predict(df_test.drop(columns=["customer_ID", "S_2", "target"], axis=1))

In [36]:
# Combine predicted and actual values in same Dataframe
p = preds[["pred"]]
p = p.join(df_test["target"]).rename(columns={"target": "actual"})
p.head()

,pred,actual
0,1,1
1,1,1
2,0,0
3,1,1
4,1,1


In [37]:
# Print the accurate prediction count
print(f"{(p.pred==p.actual).astype(int).sum()}/{len(p)} are correct")

5011/6000 are correct


In [38]:
print(f"{p[(p['pred']==1) & (p['actual']==1)].shape[0]}/1500 defaulters are correct")

1346/1500 defaulters are correct


In [39]:
print(f"{p[(p['pred']==0) & (p['actual']==0)].shape[0]}/4500 non-defaulters are correct")

3665/4500 non-defaulters are correct
